In [1]:
import pandas as pd
import os
import missingno as msno

In [2]:
wi_zipcode = pd.read_csv("assets\zipcode.csv")

In [3]:
house_price_folder_path = "D:\\Dropbox (University of Michigan)\\Dropbox (University of Michigan)\\696\\assets\\house_price"
os.chdir(house_price_folder_path)

df = pd.DataFrame([])

# Read in the data and concatenate into a single dataframe
for root, dirs, files in os.walk("."):
    for name in files:
        df_temp = pd.read_csv(name, encoding = "ISO-8859-1")
        df = pd.concat([df, df_temp], axis=0)

# Save the data to a csv file
df.to_csv("house_price.csv", index=False)

C:\Users\aliso\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (79) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\aliso\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (78) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\aliso\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (78,84) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\aliso\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (68,78) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, 

In [4]:
# Read csv files
house_price_df = pd.read_csv("house_price.csv")
house_price_df.head()

C:\Users\aliso\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (0,7,8,18,19,24,29,30,31,32,33,34,35,36,38,40,41,42,43,44,45,51,52,53,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,89) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,SaleNumber,YearCaptured,GrantorType,GrantorGranteeRelationship,WeatherStds,EnergyExclusion,Section,Township,Range,PropertyType,...,OwnerInterestOtherNote,GrantorRightsOtherNotes,PreviousDocumentNumber,ConveyanceCodeOtherNote,MiscCountyTVC,MultiTVSs,WaterFrontFeet,VolumeJacket,Unnamed: 89,lotsize_sqft
0,201700039900001,2017,4.0,1.0,2.0,3.0,NaN,NaN,NaN,2,...,NaN,NaN,31O9I,NaN,NaN,2.0,0.0,NaN,NaN,NaN
1,201700039900003,2016,1.0,6.0,2.0,3.0,NaN,NaN,NaN,2,...,NaN,NaN,NaN,NaN,NaN,2.0,0.0,NaN,NaN,NaN
2,201700039900005,2016,4.0,1.0,2.0,7.0,NaN,NaN,NaN,2,...,NaN,NaN,NaN,NaN,NaN,2.0,0.0,NaN,NaN,NaN
3,201700039900014,2016,1.0,1.0,2.0,1.0,NaN,NaN,NaN,2,...,NaN,NaN,NaN,NaN,NaN,2.0,0.0,NaN,NaN,NaN
4,201700039900016,2016,1.0,1.0,2.0,7.0,NaN,NaN,NaN,2,...,NaN,NaN,NaN,NaN,NaN,2.0,0.0,NaN,NaN,NaN


In [5]:
house_price_df.columns

Index(['SaleNumber', 'YearCaptured', 'GrantorType',
       'GrantorGranteeRelationship', 'WeatherStds', 'EnergyExclusion',
       'Section', 'Township', 'Range', 'PropertyType', 'PredominateUse',
       'MultiFamilyUnits', 'AgrOwnerLessThan5years', 'TotalAcres',
       'WaterFrontIndicator', 'TransferType', 'OwnerInterestTransferred',
       'GrantorRightsRetained', 'PersPropertyValueExcluded',
       'PersPropertyValueExempt', 'TotalRealEstateValue', 'TransferFee',
       'TransferExemptionNumber', 'FinancingCode', 'DocumentNumber',
       'DateRecorded', 'DateConveyed', 'DeedDate', 'ConveyanceCode',
       'ParcelIdentification', 'MultiGrantors', 'GrantorLastName',
       'GrantorFirstName', 'GrantorStreetNumber', 'GrantorAddress',
       'GrantorCity', 'GrantorState', 'GrantorZip', 'CertificationDate',
       'MultiGrantees', 'GranteeLastName', 'GranteeFirstName',
       'GranteeStreetNumber', 'GranteeAddress', 'GranteeCity', 'GranteeState',
       'GranteeZip', 'GranteeCertificatio

In [6]:
updated_house_price = house_price_df[['SaleNumber', 'YearCaptured', 'PropertyType', 'PredominateUse',
       'MultiFamilyUnits', 'TransferType', 'TotalRealEstateValue', 'TransferFee',
       'TransferExemptionNumber', 'TVCname', 'CountyName', 'PropertyAddress', 'DateRecorded', 'LotSize1', 'LotSize', 'TotalAcres']].copy()
updated_house_price.head()

,SaleNumber,YearCaptured,PropertyType,PredominateUse,MultiFamilyUnits,TransferType,TotalRealEstateValue,TransferFee,TransferExemptionNumber,TVCname,CountyName,PropertyAddress,DateRecorded,LotSize1,LotSize,TotalAcres
0,201700039900001,2017,2,1,2,5,"$95,000.00",$0.00,3,"STEVENS POINT, CITY OF",PORTAGE,2021 - 2023 DIVISION STREET,1032017,86,86.0,0.0
1,201700039900003,2016,2,1,3,5,"$200,000.00",$0.00,16,"DARLINGTON, CITY OF",LAFAYETTE,"145-155 W, RIVER STREET",1032017,0,0.0,1.0
2,201700039900005,2016,2,5,0,5,"$4,823,200.00",$0.00,12,"KIMBERLY, VILLAGE OF",OUTAGAMIE,N. MAIN STREET,1032017,0,0.0,92.0
3,201700039900014,2016,2,1,0,1,"$143,500.00",$430.50,NaN,"APPLETON, CITY OF",OUTAGAMIE,"2624 FAIRFIELD COURT, APPLETON, WI 54911",1032017,0,0.0,0.0
4,201700039900016,2016,2,2,0,1,"$900,000.00","$2,700.00",NaN,"APPLETON, CITY OF",OUTAGAMIE,1400 WEST COLLEGE AVENUE,1032017,251,251.0,0.0


In [7]:
updated_house_price['TotalRealEstateValue'] = updated_house_price['TotalRealEstateValue'].replace(',', '')
updated_house_price['TransferFee'] = updated_house_price['TransferFee'].replace(',', '')
# convert string data type to numeric data type
updated_house_price['LotSize1'].astype('float64')
updated_house_price['LotSize'].astype('float64')
updated_house_price['TotalAcres'].astype('float64')
updated_house_price['lotsize_sqft'] = updated_house_price['LotSize1'] * updated_house_price['LotSize']
# keep family houses only
updated_house_price = updated_house_price[updated_house_price['PredominateUse'] == 1]
# remove land only real estate
updated_house_price = updated_house_price[updated_house_price['PropertyType'] != 1]
# keep single family houses only
updated_house_price = updated_house_price[updated_house_price['MultiFamilyUnits'] == 0]
# keep transfer type Original Sale only
updated_house_price = updated_house_price[updated_house_price['TransferType'] == 1]

updated_house_price.loc[updated_house_price['TotalAcres'] == 0, 'TotalAcres'] = updated_house_price['lotsize_sqft'] / 43560
updated_house_price.loc[updated_house_price['lotsize_sqft'] == 0, 'lotsize_sqft'] = updated_house_price['TotalAcres'] * 43560
# get city name from TVCname
updated_house_price['city'] = updated_house_price['TVCname'].str.split(',').str[0]


updated_house_price.head()

,SaleNumber,YearCaptured,PropertyType,PredominateUse,MultiFamilyUnits,TransferType,TotalRealEstateValue,TransferFee,TransferExemptionNumber,TVCname,CountyName,PropertyAddress,DateRecorded,LotSize1,LotSize,TotalAcres,lotsize_sqft
3,201700039900014,2016,2,1,0,1,"$143,500.00",$430.50,NaN,"APPLETON, CITY OF",OUTAGAMIE,"2624 FAIRFIELD COURT, APPLETON, WI 54911",1032017,0,0.0,0.0,0.0
7,201700039900036,2016,2,1,0,1,"$31,000.00",$0.00,2,"KENOSHA, CITY OF",KENOSHA,5903 20TH AVENUE,1032017,0,0.0,0.0,0.0
20,201700039900074,2016,2,1,0,1,"$156,900.00",$470.70,NaN,"EMPIRE, TOWN OF",FOND DU LAC,N6612 7 HILLS ROAD,1032017,0,0.0,2.0,0.0
23,201700039900082,2016,2,1,0,1,"$45,000.00",$135.00,NaN,"FOND DU LAC, CITY OF",FOND DU LAC,282 BREISTER AVE,1032017,0,0.0,0.0,0.0
28,201700039900098,2016,2,1,0,1,"$92,000.00",$276.00,NaN,"FOND DU LAC, CITY OF",FOND DU LAC,505 EMMA STREET FOND DU LAC WI 54935,1032017,0,0.0,1.0,0.0


In [13]:
zero_counts_totalacres = (updated_house_price['TotalAcres'] == 0).sum()
zero_counts_lotsize = (updated_house_price['lotsize_sqft'] == 0).sum()

print(f'Total houses: {len(updated_house_price)}')
print(f'Count of zeros in TotalAcres: {zero_counts_totalacres}')
print(f'Count of zeros in lotsize_sqft: {zero_counts_lotsize}')
updated_house_price = updated_house_price[updated_house_price['lotsize_sqft'] != 0]
updated_house_price.head()

Total houses: 841511
Count of zeros in TotalAcres: 467973
Count of zeros in lotsize_sqft: 467973


In [10]:
wi_zipcode.head()

,zip,Zipcode name,City,State,County Name
0,54405,"ABBOTSFORD, WI",ABBOTSFORD,WI,CLARK
1,54101,"ABRAMS, WI",ABRAMS,WI,OCONTO
2,53910,"ADAMS, WI",ADAMS,WI,ADAMS
3,53001,"ADELL, WI",ADELL,WI,SHEBOYGAN
4,53501,"AFTON, WI",AFTON,WI,ROCK


In [11]:
updated_wi_zipcode = wi_zipcode[['zip', 'City']].copy()
# Remove nan values
updated_wi_zipcode.dropna(inplace=True)
updated_wi_zipcode['zip'].astype('int64')
updated_wi_zipcode.head()

,zip,City
0,54405,ABBOTSFORD
1,54101,ABRAMS
2,53910,ADAMS
3,53001,ADELL
4,53501,AFTON


In [12]:
# Add a zipcode column to the house price dataframe
updated_house_price['zip'] = updated_house_price.merge(updated_wi_zipcode, how='left', left_on='city', right_on='City')['zip']
updated_house_price.drop(['TVCname', 'DateRecorded', 'LotSize1', 'LotSize', 'TotalAcres', 'city'], axis=1)

,SaleNumber,YearCaptured,PropertyType,PredominateUse,MultiFamilyUnits,TransferType,TotalRealEstateValue,TransferFee,TransferExemptionNumber,TVCname,CountyName,PropertyAddress,DateRecorded,LotSize1,LotSize,TotalAcres,lotsize_sqft,city,zip
3,201700039900014,2016,2,1,0,1,"$143,500.00",$430.50,NaN,"APPLETON, CITY OF",OUTAGAMIE,"2624 FAIRFIELD COURT, APPLETON, WI 54911",1032017,0,0.0,0.0,0.0,APPLETON,54914
7,201700039900036,2016,2,1,0,1,"$31,000.00",$0.00,2,"KENOSHA, CITY OF",KENOSHA,5903 20TH AVENUE,1032017,0,0.0,0.0,0.0,KENOSHA,53141
20,201700039900074,2016,2,1,0,1,"$156,900.00",$470.70,NaN,"EMPIRE, TOWN OF",FOND DU LAC,N6612 7 HILLS ROAD,1032017,0,0.0,2.0,87120.0,EMPIRE,54936
23,201700039900082,2016,2,1,0,1,"$45,000.00",$135.00,NaN,"FOND DU LAC, CITY OF",FOND DU LAC,282 BREISTER AVE,1032017,0,0.0,0.0,0.0,FOND DU LAC,54936
28,201700039900098,2016,2,1,0,1,"$92,000.00",$276.00,NaN,"FOND DU LAC, CITY OF",FOND DU LAC,505 EMMA STREET FOND DU LAC WI 54935,1032017,0,0.0,1.0,43560.0,FOND DU LAC,54937


In [35]:
final_house_df = updated_house_price[['SaleNumber', 'MultiFamilyUnits', 'TotalRealEstateValue', 'TransferFee', 'zip', 'lotsize_sqft']].copy()
final_house_df['TotalRealEstateValue'] = final_house_df['TotalRealEstateValue'].str[1:]
final_house_df['TransferFee'] = final_house_df['TransferFee'].str[1:]
final_house_df['TotalRealEstateValue'] = final_house_df['TotalRealEstateValue'].str.replace(',', '').astype('float64')
final_house_df['TransferFee'] = final_house_df['TransferFee'].str.replace(',', '').astype('float64')
final_house_df.reset_index(drop=True, inplace=True)
final_house_df.head()

,SaleNumber,MultiFamilyUnits,TotalRealEstateValue,TransferFee,zip,lotsize_sqft
0,201700039900074,0,156900.0,470.7,54936,87120.0
1,201700039900098,0,92000.0,276.0,54937,43560.0
2,201700039900107,0,129900.0,389.7,53402,43560.0
3,201700039900124,0,5000.0,15.0,53407,43560.0
4,201700039900127,0,205000.0,615.0,53408,43560.0


In [37]:
final_house_df.to_csv("final_house_df.csv")